In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.metrics import roc_auc_score

In [ ]:
dataset = pd.read_csv('/content/WA_Fn-UseC_-Telco-Customer-Churn.csv')

# **Exploratory Data Analysis**

In [ ]:
dataset.head()

In [ ]:
dataset.shape

In [ ]:
#Checking if there are any null values
dataset.isnull().sum()

In [ ]:
dataset.info()

In [ ]:
#Descirbe the dataset
dataset.describe()